In [2]:
import numpy as np
import os
import cv2
import time
from collections import deque
from tensorflow.keras.models import load_model
from PIL import Image, ImageTk
import tkinter as tk

class VideoApp:
    def __init__(self, root, video_path, model_path):
        self.root = root
        self.root.title("Processed Frame")
        
        # Load model
        print("Loading model ...")
        self.model = load_model(model_path)

        # Video Capture
        self.vs = cv2.VideoCapture(video_path)
        if not self.vs.isOpened():
            raise Exception("Error: Unable to open video file.")

        self.label = tk.Label(root)
        self.label.pack()

        # Start updating frames
        self.update_frame()

    def update_frame(self):
        grabbed, frame = self.vs.read()
        if not grabbed:
            print("End of video or frame could not be read.")
            self.vs.release()
            self.root.quit()
            return

        if frame is None:
            print("Error: Captured frame is empty.")
            self.root.quit()
            return

        # Process frame
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        resized_frame = cv2.resize(frame_rgb, (128, 128)).astype("float32") / 255.0
        preds = self.model.predict(np.expand_dims(resized_frame, axis=0))[0]

        # Label for violence detection
        label = (preds > 0.50)[0]
        text_color = (255, 0, 0) if label else (0, 255, 0)
        text = f"Violence: {label}"

        FONT = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame_rgb, text, (35, 50), FONT, 1.25, text_color, 3)

        # Convert frame for Tkinter
        img = Image.fromarray(frame_rgb)
        img = ImageTk.PhotoImage(img)

        # Update Tkinter Label
        self.label.config(image=img)
        self.label.image = img

        # Schedule next update
        self.root.after(50, self.update_frame)

# File Paths
video_path = r"D:\BE Project\violent_bus.mp4"
model_path = r"D:\BE Project\modelnew.h5"

# Run GUI
root = tk.Tk()
app = VideoApp(root, video_path, model_path)
root.mainloop()


Loading model ...
1/1 [==============================] - 0s 36ms/step
